# ¡Ay, Dios!

Si has llegado hasta aquí, es porque todavía tienes problemas para configurar tu entorno. ¡Lo siento mucho! No te rindas y en poco tiempo lo tendremos listo y funcionando.

Configurar un entorno de ciencia de datos puede ser un desafío porque hay mucho que hacer. Pero lo lograremos.

Y recuerda: estoy disponible para ayudarte. Envíame un mensaje o un correo electrónico a ed@edwarddonner.com y me pondré manos a la obra.

# Paso 1

Intenta ejecutar la siguiente celda (haz clic en la celda que está debajo de esta y presiona shift+return).

Si esto arroja un error, entonces es probable que no estés ejecutando en un entorno "activado". Vuelve a consultar la Parte 5 de la guía de CONFIGURACIÓN para [PC](../SETUP-PC.md) o [Mac](../SETUP-mac.md) para configurar el entorno de Anaconda (o virtualenv) y activarlo, antes de ejecutar `jupyter lab`.

Si miras en el indicador de Anaconda (PC) o en la Terminal (Mac), deberías ver `(llms)` en el indicador donde inicias `jupyter lab` - esa es la pista de que el entorno llms está activado.

Si se encuentra en un entorno activado, lo siguiente que debe intentar es reiniciar todo:
1. Cierre todas las ventanas de Jupyter, de esta manera
2. Salga de todos los indicadores de comando/Terminales/Anaconda
3. Repita la Parte 5 de las instrucciones de CONFIGURACIÓN para comenzar un nuevo entorno activado e iniciar el laboratorio de Jupyter
4. Menú del kernel >> Reiniciar kernel y borrar las salidas de todas las celdas
5. Regrese a este cuaderno y vuelva a intentar la celda a continuación.

Si **eso** no funciona, ¡comunícate conmigo en el foro del curso! Le responderé rápidamente y lo resolveremos.

In [1]:
# Esto debería ejecutarse sin salida, sin errores de importación.
# Los errores de importación podrían indicar que iniciaste Jupyter Lab sin tu entorno activado. Consulta la parte 5 de CONFIGURACIÓN.

from openai import OpenAI

# Paso 2

Comprobemos que el archivo .env existe y que la clave OpenAI está configurada correctamente.
Ejecute este código y compruebe que muestra un mensaje de éxito; de lo contrario, siga las instrucciones.

Tenga en cuenta que el archivo `.env` no aparecerá en el explorador de archivos de Jupyter Lab, porque Jupyter oculta los archivos que comienzan con un punto para su seguridad; se consideran archivos ocultos. Si necesita cambiar el nombre, deberá utilizar una terminal de comandos o el Explorador de archivos (PC) / Ventana del Finder (Mac). ¡Pregunte a ChatGPT si eso le está dando problemas o envíeme un correo electrónico!

In [3]:
from pathlib import Path

parent_dir = Path("..")
env_path = parent_dir / ".env"

if env_path.exists() and env_path.is_file():
    print("Fichero .env encontrado.")

    # Lee el contenido del fichero .env
    with env_path.open("r") as env_file:
        contents = env_file.readlines()

    key_exists = any(line.startswith("OPENAI_API_KEY=") for line in contents)
    good_key = any(line.startswith("OPENAI_API_KEY=sk-proj-") for line in contents)
    
    if key_exists and good_key:
        print("¡ÉXITO! Se encontró OPENAI_API_KEY y tiene el prefijo correcto")
    elif key_exists:
        print("Se encontró una OPENAI_API_KEY aunque no tenía el prefijo esperado sk-proj- \nVerifique nuevamente su clave en el archivo.")
    else:
        print("No encontré un OPENAI_API_KEY en el archivo .env")
else:
    print("El archivo .env no se encuentra en el directorio llm_engineering. Debe tener exactamente el nombre: .env")
    
    possible_misnamed_files = list(parent_dir.glob("*.env*"))
    
    if possible_misnamed_files:
        print("\nAdvertencia: No se encontró ningún archivo '.env', pero se encontraron los siguientes archivos en el directorio llm_engineering que contienen '.env' en el nombre. ¿Quizás sea necesario cambiar el nombre?")
        for file in possible_misnamed_files:
            print(file.name)

Fichero .env encontrado.
¡ÉXITO! Se encontró OPENAI_API_KEY y tiene el prefijo correcto


# Paso 3

Ahora, verifiquemos que la clave API esté configurada correctamente en el archivo `.env`.
Intente ejecutar la siguiente celda.

In [ ]:
# Esto debería imprimir tu clave API en la salida

import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No se encontró ninguna clave API: intente con el menú Kernel >> Reiniciar Kernel y borrar las salidas de todas las celdas")
elif api_key[:8]!="sk-proj-":
    print("Se encontró una clave API, pero no empieza por sk-proj-; verifique que esté usando la clave correcta")
elif api_key.strip() != api_key:
    print("Se encontró una clave API, pero parece que podría tener espacios o caracteres de tabulación al principio o al final; elimínelos")
else:
    print("Se encontró la clave API y hasta ahora parece buena!")

print("Mi clave es", os.getenv("OPENAI_API_KEY"))

## Debería imprimir algo como:

`Mi clave es sk-proj-blahblahblah`

Si no imprimió una clave, entonces no puede encontrar un archivo llamado `.env` en la carpeta `llm_engineering`.
El nombre del archivo debe ser exactamente `.env` - no funcionará si se llama `my-keys.env` o `.env.doc`.
Vuelva a verificar esos pasos en las instrucciones. ¿Es posible que `.env` en realidad se llame `.env.txt`? En Windows, es posible que deba cambiar una configuración en el Explorador de archivos para asegurarse de que se muestren las extensiones de archivo ("Mostrar extensiones de archivo" configurado en "Activado"). También debería ver las extensiones de archivo si escribe `dir` en el directorio `llm_engineering`.

Problemas desagradables a tener en cuenta:
- En el archivo .env, no debe haber espacio entre el signo igual y la clave. Por ejemplo: `OPENAI_API_KEY=sk-proj-...`
- Si copiaste y pegaste tu clave API desde otra aplicación, asegúrate de que no haya reemplazado los guiones de tu clave con guiones largos.
- Si cambiaste tu archivo .env, es posible que tengas que reiniciar tu "kernel" de Jupyter (el proceso de Python) para que se aplique el cambio a través del menú Kernel >> Reiniciar kernel, y luego vuelve a ejecutar las celdas desde arriba.

En el peor de los casos, si no puedes hacer que esta parte funcione, no es un gran problema. Solo tendrás que pegar tu clave en Jupyter Notebook (mira un ejemplo a continuación) y asegúrate de eliminarla antes de compartir Notebook con alguien más.

# Paso 4

Ahora ejecute el código a continuación y verá que GPT puede realizar operaciones aritméticas básicas.

Si no es así, consulte la celda a continuación.

In [ ]:
from openai import OpenAI

# O BIEN:
my_api_key = os.getenv("OPENAI_API_KEY")

# O si no ha podido hacer que .env funcione, descomente la siguiente línea y pegue su clave dentro de las comillas
# my_api_key = "REEMPLACE ESTE TEXTO CON SU CLAVE DE API DE OPENAI DENTRO DE LAS COMILLAS - debería comenzar con sk-proj-"

print(f"Using API key {my_api_key}")

openai = OpenAI(api_key=my_api_key)
completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role":"user", "content": "What's 2+2?"}],
)
print(completion.choices[0].message.content)

# Si la clave se configuró correctamente y esto sigue sin funcionar, tal vez con un error de RateLimit

¡Entonces hay algo mal con tu clave API!

Primero revisa esta página web para asegurarte de que tienes un saldo de crédito positivo.
OpenAI requiere que tengas un saldo de crédito positivo y tiene mínimos. Mi argumento de venta para OpenAI es que vale la pena para tu educación: por menos del precio de un álbum de música, desarrollarás mucha experiencia comercial valiosa. Pero no es un requisito para este curso en absoluto; puedes verme ejecutar el código de OpenAI y luego esperar hasta que lleguemos a los modelos de código abierto en la semana 3. Además, te mostraré cómo usar Ollama para ejecutar modelos de código abierto localmente.

https://platform.openai.com/settings/organization/billing/overview

También intenta crear una nueva clave (botón en la parte superior derecha) aquí:

https://platform.openai.com/api-keys

A veces, OpenAI puede tardar unos minutos en darte acceso después de que lo intentes.

## Si todo lo demás falla:

(1) ¡Intenta pegar tu error en ChatGPT o Claude! Es sorprendente la frecuencia con la que pueden resolver las cosas.

(2) ¡Contáctame! En el foro del curso

¡Muchas gracias y lamento que esto te esté causando problemas!